In [ ]:

# 📊 Colab-Ready Employer Matching Script
# Author: Will Draney
# Purpose: Clean and match donor/employer names using fuzzy logic

import pandas as pd
from fuzzywuzzy import fuzz
import re
import string
from google.colab import files
import io

# Upload CSV
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

# Cleaning function
def clean_name(name):
    if not isinstance(name, str):
        return ""
    name = name.lower()
    name = name.translate(str.maketrans("", "", string.punctuation))
    suffixes = [' inc', ' corp', ' corporation', ' llc', ' ltd', ' limited',
                ' company', ' co', ' incorporated', ' group', ' holdings']
    for suffix in suffixes:
        name = re.sub(suffix + r"$", "", name)
        name = re.sub(suffix + r" ", " ", name)
    return " ".join(name.split())

# Apply cleaning
df['cleaned_employer'] = df['EMPLOYERorOCCUPATION'].apply(clean_name)
df['cleaned_exec_employer'] = df['exec_employer'].apply(clean_name)

# Apply fuzzy matching
df['match_score'] = df.apply(
    lambda row: fuzz.token_sort_ratio(row['cleaned_employer'], row['cleaned_exec_employer'])
    if row['cleaned_employer'] and row['cleaned_exec_employer'] else 0,
    axis=1
)

# Flag matches
df['match_status'] = df['match_score'].apply(lambda x: 'Match' if x >= 90 else 'Needs Review')

# Preview results
df[['NAME', 'EMPLOYERorOCCUPATION', 'exec_fullname', 'exec_employer',
    'match_score', 'match_status']].head(10)

# Save output
df.to_csv('matched_output.csv', index=False)
print("✅ Matching complete. File saved as 'matched_output.csv'")
